
# Exoplanet ML — Ensamble + Umbral para Bajar Falsos Positivos

**Fecha:** 2025-10-04 19:40:37

Este notebook unifica la evaluación de **RandomForest**, **ExtraTrees**, **AdaBoost** y un **Stacking** (LogReg meta) con un **mismo protocolo CV estratificado**.  
Incluye:
- **OOF evaluation** con `cross_val_predict` para métricas estables (PR-AUC, ROC-AUC, precisión, recall, F1).
- **Curva Precisión–Recall** y **selección de umbral** para controlar falsos positivos (priorizar precisión mínima).
- **Calibración** de probabilidades (`CalibratedClassifierCV`) antes de fijar el umbral.
- Exportación del **modelo final + umbral** y un `registry.json` con todas las métricas.

> **TODO**: Completá las rutas/columnas de tus datos en la celda *Carga de datos*.


In [ ]:

# ==== Setup & reproducibilidad ====
import os, warnings, json, joblib
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_recall_curve,
                             confusion_matrix, precision_score, recall_score, f1_score, roc_curve)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
np.random.seed(42)

# Directorios
MODELS_DIR = Path("models")
MODELS_DIR.mkdir(exist_ok=True)
REGISTRY_PATH = MODELS_DIR / "registry.json"


## Carga de datos

In [ ]:

# ==== Carga de datos ====
# OPCIÓN A (CSV): descomentar y definir
# DATA_PATH = "data/dataset.csv"
# TARGET_COL = "label"
# df = pd.read_csv(DATA_PATH)
# y = df[TARGET_COL].astype(int).values
# feature_cols = [c for c in df.columns if c != TARGET_COL]
# X = df[feature_cols].values

# OPCIÓN B (X, y ya construidos en memoria): si ejecutás el notebook dentro de tu proyecto, podés
# simplemente asignar X, y antes y saltar esta celda.

# OPCIÓN C (usar features.json para columnas) — si lo tenés disponible:
feature_cols = None
if Path("/mnt/data/features.json").exists():
    try:
        with open("/mnt/data/features.json", "r") as f:
            feature_cols = json.load(f)
            print("features.json detectado con", len(feature_cols), "features.")
    except Exception as e:
        print("No se pudo leer features.json:", e)

# Si ya tenés X, y en el entorno, dejá esta celda como informativa:
try:
    X
    y
    print("X,y ya existen. Shape:", np.shape(X), "y positivos:", int(np.sum(y)))
    if feature_cols is not None and isinstance(X, np.ndarray) and X.shape[1] == len(feature_cols):
        print("Las columnas de features.json coinciden con X.shape[1].")
except NameError:
    print(">> Definí X,y o cargalos desde CSV. Ver instrucciones arriba.")


## Protocolo de evaluación y utilidades

In [ ]:

# ==== Protocolo de evaluación ====
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def pr_metrics_from_probs(y_true, y_prob, threshold=0.5):
    ap = average_precision_score(y_true, y_prob)  # PR-AUC
    roc = roc_auc_score(y_true, y_prob)
    y_hat = (y_prob >= threshold).astype(int)
    prec = precision_score(y_true, y_hat, zero_division=0)
    rec  = recall_score(y_true, y_hat, zero_division=0)
    f1   = f1_score(y_true, y_hat, zero_division=0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_hat).ravel()
    return {
        "PR_AUC": ap, "ROC_AUC": roc, "precision@thr": prec, "recall@thr": rec, "F1@thr": f1,
        "TP": int(tp), "FP": int(fp), "FN": int(fn), "TN": int(tn), "threshold": float(threshold)
    }

def pick_threshold_for_min_precision(y_true, y_prob, min_precision=0.90):
    prec, rec, thr = precision_recall_curve(y_true, y_prob)
    mask = prec >= min_precision
    if mask.any():
        thr_ok = thr[mask].min()  # menor umbral que cumple la precisión mínima
    else:
        thr_ok = 0.5  # fallback
    return float(thr_ok)

def plot_pr_curves(models_probs, y_true):
    plt.figure(figsize=(6,5))
    for name, probs in models_probs.items():
        prec, rec, _ = precision_recall_curve(y_true, probs)
        ap = average_precision_score(y_true, probs)
        plt.plot(rec, prec, label=f"{name} (AP={ap:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Curva Precisión–Recall (OOF)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


## Definición de modelos

In [ ]:
# ==== OPCIONAL: Cargar modelos ya entrenados desde archivos .pkl ====
# Si querés ahorrar tiempo y ya tenés tus modelos entrenados, descomentá este bloque.

import joblib
from pathlib import Path

PRETRAINED_DIR = Path("/mnt/data")  # o "models" si los moviste allí

pretrained_files = {
    "RandomForest": PRETRAINED_DIR / "models/random_forest.pkl",
    "ExtraTrees": PRETRAINED_DIR / "models/extra_trees.pkl",
    "AdaBoost": PRETRAINED_DIR / "models/adaboost.pkl",
    "Stacking": PRETRAINED_DIR / "models/stacking.pkl",
}

loaded = {}
for name, path in pretrained_files.items():
    if path.exists():
        try:
            model = joblib.load(path)
            loaded[name] = model
            print(f"✅ Cargado modelo preentrenado: {name}")
        except Exception as e:
            print(f"⚠️ No se pudo cargar {name}: {e}")
    else:
        print(f"⚠️ Archivo no encontrado para {name}: {path}")

if loaded:
    models = loaded  # sobrescribe los modelos definidos más arriba
    print("\\nUsando modelos preentrenados para evaluación.")
else:
    print("\\nNo se encontraron modelos preentrenados, se usarán los definidos arriba.")


: 

In [ ]:

# ==== Modelos base ====
rf  = RandomForestClassifier(n_estimators=600, criterion='entropy', random_state=42, n_jobs=-1, class_weight=None)
et  = ExtraTreesClassifier(n_estimators=600, criterion='entropy', random_state=42, n_jobs=-1)
ada = AdaBoostClassifier(n_estimators=600, learning_rate=0.1, random_state=42)

# Meta-modelo para stacking
meta = LogisticRegression(max_iter=2000, n_jobs=None)

stack = StackingClassifier(
    estimators=[('rf', rf), ('et', et), ('ada', ada)],
    final_estimator=meta,
    cv=cv,
    passthrough=False,
    n_jobs=-1
)

models = {
    "RandomForest": rf,
    "ExtraTrees": et,
    "AdaBoost": ada,
    "Stacking": stack
}

print("Modelos definidos:", list(models.keys()))


## Evaluación OOF (mismo CV)

In [ ]:

# ==== Evaluación OOF ====
try:
    X; y
except NameError:
    raise RuntimeError("Definí X,y antes de correr esta celda. Ver 'Carga de datos'.")

oof_probs = {}
scores_tbl = []

for name, clf in models.items():
    # Si tu pipeline necesitara escalado, encadenalo aquí:
    pipe = Pipeline([
        # ("scaler", StandardScaler()),  # activar si hace falta
        ("clf", clf)
    ])
    probs = cross_val_predict(pipe, X, y, cv=cv, method="predict_proba")[:,1]
    oof_probs[name] = probs
    metrics = pr_metrics_from_probs(y, probs, threshold=0.5)
    row = {"model": name, **metrics}
    scores_tbl.append(row)
    print(f"{name}  -> PR-AUC={metrics['PR_AUC']:.4f} | ROC-AUC={metrics['ROC_AUC']:.4f} | "
          f"Prec@0.5={metrics['precision@thr']:.3f} | Recall@0.5={metrics['recall@thr']:.3f}")

scores_df = pd.DataFrame(scores_tbl).sort_values("PR_AUC", ascending=False).reset_index(drop=True)
scores_df


### Curvas PR (comparación OOF)

In [ ]:

plot_pr_curves(oof_probs, y_true=y)


## Selección de modelo + umbral

In [ ]:

# Elegimos el mejor por PR-AUC (primer fila del scores_df)
best_name = scores_df.loc[0, "model"]
print("Mejor por PR-AUC (OOF):", best_name)

# Umbral para cumplir precisión mínima (ajustá a tu objetivo)
MIN_PRECISION = 0.90
thr_best = pick_threshold_for_min_precision(y, oof_probs[best_name], min_precision=MIN_PRECISION)
print(f"Umbral elegido para {best_name} con precisión ≥ {MIN_PRECISION:.2f}: {thr_best:.3f}")

# Métricas en ese punto de operación
op_metrics = pr_metrics_from_probs(y, oof_probs[best_name], threshold=thr_best)
pd.DataFrame([op_metrics])


## Calibración y entrenamiento final (full data)

In [ ]:

# Calibramos el mejor modelo y lo entrenamos en el 100% de los datos
base = models[best_name]
pipe = Pipeline([
    # ("scaler", StandardScaler()),  # activar si hace falta para tus features
    ("clf", base)
])

calibrated = CalibratedClassifierCV(pipe, method="isotonic", cv=cv)
calibrated.fit(X, y)

# Guardado
final_model_path = MODELS_DIR / f"{best_name.lower()}_calibrated.pkl"
joblib.dump(calibrated, final_model_path)
print("✅ Modelo calibrado guardado en:", final_model_path)

# Guardamos parámetros operativos (umbral + columnas)
export = {
    "model_name": best_name,
    "threshold": float(thr_best),
    "feature_cols": feature_cols if feature_cols is not None else (list(range(X.shape[1])) if hasattr(X, 'shape') else None),
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "min_precision_target": float(MIN_PRECISION)
}
with open(MODELS_DIR / "inference_config.json", "w") as f:
    json.dump(export, f, indent=2)
print("✅ inference_config.json guardado.")


## Registry de experimentos

In [ ]:

# Actualizamos/creamos registry.json con resultados OOF y punto de operación
registry_entry = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "cv": {"n_splits": 5, "shuffle": True, "random_state": 42},
    "scores": scores_df.to_dict(orient="records"),
    "selected_model": best_name,
    "operating_point": {
        "threshold": float(thr_best),
        **pr_metrics_from_probs(y, oof_probs[best_name], threshold=thr_best)
    },
    "artifacts": {
        "model_path": str(final_model_path),
        "inference_config": str(MODELS_DIR / "inference_config.json")
    }
}

if REGISTRY_PATH.exists():
    with open(REGISTRY_PATH, "r") as f:
        try:
            reg = json.load(f)
            if not isinstance(reg, list): reg = [reg]
        except Exception:
            reg = []
else:
    reg = []

reg.append(registry_entry)
with open(REGISTRY_PATH, "w") as f:
    json.dump(reg, f, indent=2)

print("✅ Registry actualizado en", REGISTRY_PATH)


## Inferencia con umbral

In [ ]:

# ==== Inferencia ====
def load_model_and_config(model_path=None, config_path=None):
    model_path = model_path or (MODELS_DIR / f"{best_name.lower()}_calibrated.pkl")
    config_path = config_path or (MODELS_DIR / "inference_config.json")
    model = joblib.load(model_path)
    with open(config_path, "r") as f:
        cfg = json.load(f)
    return model, cfg

def predict_with_threshold(X_new, model, threshold):
    probs = model.predict_proba(X_new)[:,1]
    return (probs >= threshold).astype(int), probs

# Ejemplo de uso (descomentar cuando tengas X_new):
# model, cfg = load_model_and_config()
# y_pred, y_prob = predict_with_threshold(X_new, model, cfg["threshold"])
# print("Preds:", y_pred[:10], "Probs:", y_prob[:10])


## Housekeeping (opcional)

In [ ]:

# Renombrado sugerido para mantener consistencia (ejecutar sólo si querés limpiar)
# Ojo: no elimina nada, solo muestra propuestas.
existing = list(MODELS_DIR.glob("*.pkl"))
print("Archivos actuales en models/:")
for p in existing:
    print("-", p.name)
print("\nSugerencia: conservar sólo los modelos calibrados y registrar métricas en registry.json")
